**This is the first draft of my project!**

DATASETS: "Top Spotify songs from 2010-2019 - BY YEAR" and "Spotify - All Time Top 2000s Mega Dataset"

URLS: https://www.kaggle.com/datasets/leonardopena/top-spotify-songs-from-20102019-by-year?resource=download and https://www.kaggle.com/datasets/iamsumat/spotify-top-2000s-mega-dataset?select=Spotify-2000.csv

TASK: Improving Song Genre Classification with Text Classification.

STEP 1: Build baseline with song audio features

STEP 2: Build baseline with song lyrics

STEP 3: Combine models

STEP 4: Finetune and improve scores of 1 to 3.

Hypothesis: Lyrics does contain important information not shown in the audio itself that can improve audio classifiers.
(This is most likely already shown in some research but the process of this project seemed very intriguing to me and does contain many steps of problem solving!
)

In [20]:
import pandas as pd
import numpy as np

Data Preaparation

In [32]:
df_B42000 = pd.read_csv("./DATASETS/Spotify-2000.csv")
df_10_to_19 = pd.read_csv("./DATASETS/top10s.csv")
df_B42000['Length (Duration)'] = df_B42000['Length (Duration)'].apply(lambda x: int(x.replace(',',''))) #numbers over 1000 in this column got handled as strings.
df_B42000['Length (Duration)'] = df_B42000['Length (Duration)'].astype(float) #numvers got handled as ints so the describe function did not work for this column.


In [35]:
df_B42000.describe()

,Index,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
count,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.00000
mean,997.500000,1992.992979,120.215647,59.679539,53.238215,-9.008526,19.012036,49.408726,262.443330,28.858074,4.994985,59.52658
std,575.762538,16.116048,28.028096,22.154322,15.351507,3.647876,16.727378,24.858212,93.604387,29.011986,4.401566,14.35160
min,1.000000,1956.000000,37.000000,3.000000,10.000000,-27.000000,2.000000,3.000000,93.000000,0.000000,2.000000,11.00000
25%,499.250000,1979.000000,99.000000,42.000000,43.000000,-11.000000,9.000000,29.000000,212.000000,3.000000,3.000000,49.25000
50%,997.500000,1993.000000,119.000000,61.000000,53.000000,-8.000000,12.000000,47.000000,245.000000,18.000000,4.000000,62.00000
75%,1495.750000,2007.000000,136.000000,78.000000,64.000000,-6.000000,23.000000,69.750000,289.000000,50.000000,5.000000,71.00000
max,1994.000000,2019.000000,206.000000,100.000000,96.000000,-2.000000,99.000000,99.000000,1412.000000,99.000000,55.000000,100.00000


In [36]:
df_10_to_19.describe()

,Unnamed: 0,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
count,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000
mean,302.000000,2014.592040,118.545605,70.504146,64.379768,-5.578773,17.774461,52.225539,224.674959,14.326700,8.358209,66.520730
std,174.215384,2.607057,24.795358,16.310664,13.378718,2.798020,13.102543,22.513020,34.130059,20.766165,7.483162,14.517746
min,1.000000,2010.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,134.000000,0.000000,0.000000,0.000000
25%,151.500000,2013.000000,100.000000,61.000000,57.000000,-6.000000,9.000000,35.000000,202.000000,2.000000,4.000000,60.000000
50%,302.000000,2015.000000,120.000000,74.000000,66.000000,-5.000000,12.000000,52.000000,221.000000,6.000000,5.000000,69.000000
75%,452.500000,2017.000000,129.000000,82.000000,73.000000,-4.000000,24.000000,69.000000,239.500000,17.000000,9.000000,76.000000
max,603.000000,2019.000000,206.000000,98.000000,97.000000,-2.000000,74.000000,98.000000,424.000000,99.000000,48.000000,99.000000


Since the two datasets both are Kaggle datasets from Spotify, they use the same metrics and the values in the columns seem to follow similiar distributions it does not seem to be any problems to concatenate the datasets to one. 

In [69]:
df_10_to_19.columns = df_B42000.columns #syncing column names
df = df_10_to_19.append(df_B42000, ignore_index=True)
df.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217.0,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263.0,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200.0,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295.0,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221.0,2,4,78


Since the task is to classify the Genre of the song we can remove some columns. We keep the Title, Artist for future use, but we can remove the columns Year and Popularity since they are not audio features. The Speechiness column is a gray zone but you could consider words per minute as an audio feature so we keep it as an input for now.  

In [73]:
df_compete = df.drop(df.columns[[0,4,14]], axis=1)
df_compete.head()

,Title,Artist,Top Genre,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness
0,"Hey, Soul Sister",Train,neo mellow,97,89,67,-4,8,80,217.0,19,4
1,Love The Way You Lie,Eminem,detroit hip hop,87,93,75,-5,52,64,263.0,24,23
2,TiK ToK,Kesha,dance pop,120,84,76,-3,29,71,200.0,10,14
3,Bad Romance,Lady Gaga,dance pop,119,92,70,-4,8,71,295.0,0,4
4,Just the Way You Are,Bruno Mars,pop,109,84,64,-5,9,43,221.0,2,4
